<a href="https://colab.research.google.com/github/deacs11/CrewAI_News_Sentiment_Analyzer_For_Stocks/blob/main/CrewAI_News_Sentiment_Analyzer_For_Stocks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# @title 1. Install necessary libraries
# Installs crewai, tools, OpenAI client, Colab module, and search tool
!pip install crewai crewai-tools langchain-openai google-colab duckduckgo-search==5.3.1b1 -q

print("Library installation completed!")

Library installation completed!


In [3]:
# @title 2. Import modules and configure API keys from secrets
import os
from google.colab import userdata # To read Colab secrets
from crewai import Agent, Task, Crew, Process
from crewai_tools import SerperDevTool # Using Serper for web search
from langchain_openai import ChatOpenAI

# --- API key configuration from Colab secrets ---
print("Attempting to load API keys from Secrets...")
try:
    # Reads the OpenAI key from the secret 'OPENAI_API_KEY'
    os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
    # Reads the Serper key from the secret 'SERPER_API_KEY'
    os.environ["SERPER_API_KEY"] = userdata.get('SERPER_API_KEY')
    print("-> API key environment variables set successfully.")
except Exception as e:
    print(f"!!! Error loading API keys from secrets: {e}")
    print("!!! Make sure you have set them correctly in the Colab Secrets panel")
    print("!!! and enabled 'Notebook access' for both.")

# Debug block to verify keys
print("-" * 20)
retrieved_key = os.environ.get("OPENAI_API_KEY")
if retrieved_key:
    print(f"OpenAI key FOUND in environment: '{retrieved_key[:5]}...{retrieved_key[-4:]}'")
else:
    print("!!! OpenAI key NOT FOUND.")
print("-" * 20)
retrieved_serper_key = os.environ.get("SERPER_API_KEY")
if retrieved_serper_key:
     print(f"Serper key FOUND in environment: '{retrieved_serper_key[:5]}...{retrieved_serper_key[-4:]}'")
else:
    print("--- Serper key NOT FOUND (required for NewsFinder agent).")
print("-" * 20)

/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_config.py:323: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  warnings.warn(DEPRECATION_MESSAGE, DeprecationWarning)
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_fields.py:198: UserWarning: Field name "schema" in "DatabricksQueryToolSchema" shadows an attribute in parent "BaseModel"
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:617: UserWarning: <built-in function callable> is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this error wrap the type with `pydantic.SkipValidation`.
  warn(
/usr/local/lib/python3.11/dist-packages/crewai_t

Attempting to load API keys from Secrets...
-> API key environment variables set successfully.
--------------------
OpenAI key FOUND in environment: 'sk-pr...DKkA'
--------------------
Serper key FOUND in environment: '7354c...dc56'
--------------------


In [12]:
# @title 3. Define stock ticker & IMPORTANT disclaimer

# --- MODIFY THIS VARIABLE WITH THE STOCK TICKER SYMBOL ---
# Example: "AAPL", "GOOGL", "MSFT", "TSLA", "NVDA"
stock_ticker = "NVDA" # Example: NVIDIA
# ---------------------------------------------------------

# --- !!! CRITICAL DISCLAIMER !!! ---
print("*"*70)
print("!!! DISCLAIMER: Financial News Sentiment Analysis Assistant !!!")
print("This tool provides automated analysis of news sentiment for informational")
print("purposes ONLY. It is experimental and based on AI interpretation.")
print("It DOES NOT provide financial or investment advice.")
print("Output may contain errors, omissions, or misinterpretations of complex")
print("financial news. Market sentiment can change rapidly.")
print("All findings MUST be independently verified, and any investment")
print("decisions should be made based on your own research and consultation")
print("with a qualified financial advisor.")
print("Do not rely solely on this tool for investment decisions.")
print("*"*70)
print(f"\nStock ticker set for analysis: {stock_ticker}")

**********************************************************************
!!! DISCLAIMER: Financial News Sentiment Analysis Assistant !!!
This tool provides automated analysis of news sentiment for informational
purposes ONLY. It is experimental and based on AI interpretation.
It DOES NOT provide financial or investment advice.
Output may contain errors, omissions, or misinterpretations of complex
financial news. Market sentiment can change rapidly.
All findings MUST be independently verified, and any investment
decisions should be made based on your own research and consultation
with a qualified financial advisor.
Do not rely solely on this tool for investment decisions.
**********************************************************************

Stock ticker set for analysis: NVDA


In [13]:
# @title 4. Select LLM and initialize tools

# --- Choose the language model (LLM) ---
try:
    # GPT-4 recommended for better nuance in sentiment analysis of financial text
    llm = ChatOpenAI(model="gpt-4-turbo")
    # llm = ChatOpenAI(model="gpt-3.5-turbo") # May be less accurate on sentiment/themes
    print(f"LLM ({llm.model_name}) initialized successfully.")
except Exception as e:
    print(f"!!! Error initializing ChatOpenAI: {e}")
    print("!!! Verify OpenAI API key.")
    raise

# --- Initialize tools ---
# Web search tool is needed for the NewsFinder Agent
search_tool = None
try:
    search_tool = SerperDevTool()
    print("SerperDevTool initialized.")
except Exception as e:
    print(f"--- Warning: Error initializing SerperDevTool: {e}. News search will fail.")

available_tools = [search_tool] if search_tool else []
print(f"Tools actually available for research: {[t.name for t in available_tools] if available_tools else 'None'}")

LLM (gpt-4-turbo) initialized successfully.
SerperDevTool initialized.
Tools actually available for research: ['Search the internet with Serper']


In [14]:
# @title 5. Define agents for sentiment analysis crew

print("Defining sentiment analysis agents...")

if 'llm' not in locals() or llm is None:
     raise ValueError("LLM not initialized.")
if not available_tools:
     print("--- Warning: No search tools available. NewsFinder agent will fail.")

# --- Agent 1: News Finder ---
news_finder = Agent(
    role='Financial News Aggregator',
    goal=f'Find the 7 most recent and relevant news articles (headlines and snippets/URLs) specifically related to the company associated with the stock ticker "{stock_ticker}". Prioritize reputable financial news sources if possible.',
    backstory=(
        "You are a specialized news retrieval agent focused on financial markets. You use advanced search techniques "
        "to locate timely and pertinent news items for specific companies, filtering out irrelevant noise. "
        "You provide concise summaries or headlines for quick assessment."
    ),
    tools=available_tools, # Requires search_tool
    llm=llm,
    allow_delegation=False,
    verbose=True,
    max_iter=5
)
print("- Agent 'news_finder' defined.")

# --- Agent 2: Article Sentiment Assessor ---
article_sentiment_assessor = Agent(
    role='Financial News Sentiment Analyst',
    goal=f'Analyze the sentiment of each provided news headline/snippet regarding the company "{stock_ticker}". Classify each item as Positive, Negative, or Neutral specifically concerning the company\'s outlook, performance, or significant events mentioned. Avoid general market sentiment.',
    backstory=(
        "You are an AI analyst trained to interpret the nuances of financial news language. You understand how specific events "
        "(earnings reports, product launches, regulatory news) are typically framed and can discern the implied sentiment towards "
        "the company itself, distinguishing it from broader market commentary."
    ),
    tools=[], # Works on text provided in context
    llm=llm,
    allow_delegation=False,
    verbose=True,
    max_iter=7 # Needs iteration per news item
)
print("- Agent 'article_sentiment_assessor' defined.")

# --- Agent 3: Sentiment Aggregator ---
sentiment_aggregator = Agent(
    role='Sentiment Data Counter',
    goal='Tally the sentiment classifications (Positive, Negative, Neutral) from the list of analyzed news articles. Calculate the total count for each category and determine the overall prevailing sentiment based on the counts.',
    backstory=(
        "You are a meticulous data processor. You accurately count categorized items and provide a clear summary "
        "of the distribution, determining the majority category or indicating a mixed result if counts are close."
    ),
    tools=[],
    llm=llm,
    allow_delegation=False,
    verbose=True,
    max_iter=3
)
print("- Agent 'sentiment_aggregator' defined.")

# --- Agent 4: Key Themes Extractor ---
key_themes_extractor = Agent(
    role='News Narrative Analyst',
    goal=f'Identify the 2-3 main recurring topics or themes mentioned across the analyzed news articles that are driving the observed sentiment (positive or negative) regarding "{stock_ticker}".',
    backstory=(
        "You read between the lines of news reports to understand the underlying narrative. You connect recurring topics "
        "(like 'strong earnings', 'new product concerns', 'regulatory scrutiny', 'market share gains') to the overall sentiment picture, "
        "explaining *why* the sentiment might be leaning a certain way."
    ),
    tools=[], # Works on analyzed news context
    llm=llm,
    allow_delegation=False,
    verbose=True,
    max_iter=5
)
print("- Agent 'key_themes_extractor' defined.")

# --- Agent 5: Sentiment Brief Writer ---
sentiment_brief_writer = Agent(
    role='Financial Sentiment Reporter',
    goal='Compile a concise summary brief about the recent news sentiment for "{stock_ticker}". The brief must include: the overall sentiment assessment, the count of positive/negative/neutral articles found, the key themes driving the sentiment, and **prominent disclaimers** stating this is AI-generated informational analysis and NOT financial advice.',
    backstory=(
        "You specialize in creating clear, objective summaries of complex information for a non-expert audience, while strictly adhering "
        "to required cautionary language. You synthesize the findings from sentiment analysis and theme extraction into a brief report, "
        "always prioritizing the inclusion of necessary disclaimers about the information's nature and limitations."
    ),
    tools=[],
    llm=llm,
    allow_delegation=False,
    verbose=True,
    max_iter=3
)
print("- Agent 'sentiment_brief_writer' defined.")

print("All sentiment analysis agents defined.")

Defining sentiment analysis agents...
- Agent 'news_finder' defined.
- Agent 'article_sentiment_assessor' defined.
- Agent 'sentiment_aggregator' defined.
- Agent 'key_themes_extractor' defined.
- Agent 'sentiment_brief_writer' defined.
All sentiment analysis agents defined.


In [15]:
# @title 6. Define tasks for sentiment analysis crew

print("Defining sentiment analysis tasks...")

# Verify agents are defined
if 'news_finder' not in locals() or \
   'article_sentiment_assessor' not in locals() or \
   'sentiment_aggregator' not in locals() or \
   'key_themes_extractor' not in locals() or \
   'sentiment_brief_writer' not in locals():
    raise ValueError("One or more required agents are not defined.")

# --- Task 1: Find Recent News ---
task_find_news = Task(
    description=(
        f"Using available web search tools, find the 7 most recent and relevant news articles "
        f"(headlines, snippets, URLs) specifically about the company associated with the ticker symbol: '{stock_ticker}'. "
        f"Focus on financial news sources if possible. List the findings clearly."
    ),
    agent=news_finder,
    expected_output=(
        "A list of 7 recent news items (headline, snippet, URL) related to the stock ticker. "
        "Example: [{'title': 'Apple Beats Earnings...', 'snippet': '...', 'link': '...'}, ...]"
    )
)
print("- Task 'task_find_news' defined.")

# --- Task 2: Assess Sentiment of Articles ---
task_assess_sentiment = Task(
    description=(
        f"Analyze the list of news items provided in context (from the previous task). For each item, determine the sentiment "
        f"(Positive, Negative, or Neutral) specifically concerning the company '{stock_ticker}'. "
        f"Ignore general market sentiment unless it directly impacts the company outlook discussed in the article. "
        f"Output a list containing the original news item details along with its assessed sentiment."
    ),
    agent=article_sentiment_assessor,
    expected_output=(
        "A list of news items, where each item includes the original title/snippet/link and an added 'sentiment' field "
        "classified as 'Positive', 'Negative', or 'Neutral'. "
        "Example: [{'title': '...', 'snippet': '...', 'link': '...', 'sentiment': 'Positive'}, ...]"
    ),
    context=[task_find_news] # Needs the list of news items
)
print("- Task 'task_assess_sentiment' defined.")

# --- Task 3: Aggregate Sentiment Scores ---
task_aggregate_sentiment = Task(
    description=(
        "Process the list of news items with their assessed sentiments (provided in context). "
        "Count the total number of Positive, Negative, and Neutral articles. Determine the overall prevailing sentiment "
        "(e.g., 'Overall Positive', 'Overall Negative', 'Mixed/Neutral')."
    ),
    agent=sentiment_aggregator,
    expected_output=(
        "A summary stating the counts for each sentiment category and the determined overall sentiment. "
        "Example: 'Sentiment Analysis Results: Positive: 4, Neutral: 2, Negative: 1. Overall sentiment: Positive.'"
    ),
    context=[task_assess_sentiment] # Needs the list with sentiments
)
print("- Task 'task_aggregate_sentiment' defined.")

# --- Task 4: Extract Key Sentiment Themes ---
task_extract_themes = Task(
    description=(
        f"Review the list of news items and their sentiments (provided in context). Identify the 2-3 main recurring themes or topics "
        f"that appear to be driving the positive and/or negative sentiment regarding '{stock_ticker}'. Summarize these key themes concisely."
    ),
    agent=key_themes_extractor,
    expected_output=(
        "A short list or brief paragraph summarizing the 2-3 key themes influencing the observed sentiment. "
        "Example: 'Key themes driving sentiment: Strong Q4 earnings report (Positive), concerns about upcoming regulatory changes (Negative), successful new product adoption (Positive).'"
    ),
    context=[task_assess_sentiment] # Needs the news items and sentiments to find themes
)
print("- Task 'task_extract_themes' defined.")

# --- Task 5: Write Sentiment Brief ---
task_write_brief = Task(
    description=(
        "Compile a final sentiment brief for '{stock_ticker}' based on the aggregated sentiment results and the extracted key themes (both provided in context). "
        "The brief must clearly state: \n"
        "1. The overall sentiment assessment.\n"
        "2. The counts of Positive, Negative, and Neutral articles analyzed.\n"
        "3. The key themes identified as driving the sentiment.\n"
        "4. **CRITICAL:** Include prominent disclaimers stating that this report is AI-generated, for informational purposes only, NOT financial advice, and requires independent verification and professional consultation before making any investment decisions."
    ),
    agent=sentiment_brief_writer,
    expected_output=(
        "A concise, well-structured sentiment brief (plain text or Markdown). It MUST contain the overall sentiment, category counts, key themes, and clear, prominent disclaimers about the nature of the information and the need for professional advice. The disclaimers should be impossible to miss."
    ),
    context=[task_aggregate_sentiment, task_extract_themes] # Needs the final counts and themes
)
print("- Task 'task_write_brief' defined.")

print("All sentiment analysis tasks defined.")

Defining sentiment analysis tasks...
- Task 'task_find_news' defined.
- Task 'task_assess_sentiment' defined.
- Task 'task_aggregate_sentiment' defined.
- Task 'task_extract_themes' defined.
- Task 'task_write_brief' defined.
All sentiment analysis tasks defined.


In [16]:
# @title 7. Create and run sentiment analysis crew

print("Creating the sentiment analysis crew...")

# Verify all components are ready
if 'news_finder' not in locals() or \
   'article_sentiment_assessor' not in locals() or \
   'sentiment_aggregator' not in locals() or \
   'key_themes_extractor' not in locals() or \
   'sentiment_brief_writer' not in locals() or \
   'task_find_news' not in locals() or \
   'task_assess_sentiment' not in locals() or \
   'task_aggregate_sentiment' not in locals() or \
   'task_extract_themes' not in locals() or \
   'task_write_brief' not in locals():
    raise ValueError("Missing agents or tasks needed to create the sentiment analysis crew.")

# Assemble the Crew
sentiment_analysis_crew = Crew(
    agents=[news_finder, article_sentiment_assessor, sentiment_aggregator, key_themes_extractor, sentiment_brief_writer],
    tasks=[task_find_news, task_assess_sentiment, task_aggregate_sentiment, task_extract_themes, task_write_brief],
    process=Process.sequential,
    memory=True,
    cache=True,
    verbose=True
)

print("Sentiment analysis crew created. Starting the process with kickoff()...")

# Define initial inputs
crew_inputs = {
    'stock_ticker': stock_ticker
}

# Kick off the crew's work
result = sentiment_analysis_crew.kickoff(inputs=crew_inputs)

print("\n\n*******************************************")
print("   SENTIMENT ANALYSIS CREW FINISHED WORK!   ")
print("*******************************************")

Creating the sentiment analysis crew...
Sentiment analysis crew created. Starting the process with kickoff()...


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 9cf35596-dcdb-4f2d-b439-9b7a329cb2b2                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: 1a0c5918-a853-4475-b6e7-33b563fbb87e
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: 1a0c5918-a853-4475-b6e7-33b563fbb87e
       Status: Executing Task...
    └── 🤖 Agent: Financial News Aggregator
            Status: In Progress

# Agent: Financial News Aggregator
## Task: Using available web search tools, find the 7 most recent and relevant news articles (headlines, snippets, URLs) specifically about the company associated with the ticker symbol: 'NVDA'. Focus on financial news sources if possible. List the findings clearly.


...

🚀 Crew: crew
├── 📋 Task: 1a0c5918-a853-4475-b6e7-33b563fbb87e
│      Assigned to: Financial News Aggregator
│      Status: ✅ Completed
│   └── 🤖 Agent: Financial News Aggregator
│           Status: ✅ Completed
│       └── 🔧 Used Search the internet with Serper (2)
├── 📋 Task: 3041b407-4113-4276-a594-e963f9b9a050
│      Assigned to: Financial News Sentiment Analyst
│      Status: ✅ Completed
│   └── 🤖 Agent: Financial News Sentiment Analyst
│           Status: ✅ Completed
├── 📋 Task: 8af88eb7-a87b-4926-afda-a1a7bd401f66
│      Assigned to: Sentiment Data Counter
│      Status: ✅ Completed
│   └── 🤖 Agent: Sentiment Data Counter
│           Status: ✅ Completed
├── 📋 Task: 84f6897e-fa72-490a-a884-6500e01773ee
│      Assigned to: News Narrative Analyst
│      Status: ✅ Completed
│   └── 🤖 Agent: News Narrative Analyst
│           Status: ✅ Completed
└── 📋 Task: 9985b35e-f90d-4ae8-9254-feacef0b9c3b
       Status: Executing Task...
    └── 🤖 Agent: Financial Sentiment Reporter
            Status: ✅ Completed

🚀 Crew: crew
├── 📋 Task: 1a0c5918-a853-4475-b6e7-33b563fbb87e
│      Assigned to: Financial News Aggregator
│      Status: ✅ Completed
│   └── 🤖 Agent: Financial News Aggregator
│           Status: ✅ Completed
│       └── 🔧 Used Search the internet with Serper (2)
├── 📋 Task: 3041b407-4113-4276-a594-e963f9b9a050
│      Assigned to: Financial News Sentiment Analyst
│      Status: ✅ Completed
│   └── 🤖 Agent: Financial News Sentiment Analyst
│           Status: ✅ Completed
├── 📋 Task: 8af88eb7-a87b-4926-afda-a1a7bd401f66
│      Assigned to: Sentiment Data Counter
│      Status: ✅ Completed
│   └── 🤖 Agent: Sentiment Data Counter
│           Status: ✅ Completed
├── 📋 Task: 84f6897e-fa72-490a-a884-6500e01773ee
│      Assigned to: News Narrative Analyst
│      Status: ✅ Completed
│   └── 🤖 Agent: News Narrative Analyst
│           Status: ✅ Completed
└── 📋 Task: 9985b35e-f90d-4ae8-9254-feacef0b9c3b
       Assigned to: Financial Sentiment Reporter
       Status: ✅ Completed
    └── 🤖 Agent: Financial Sentiment Reporter
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 9985b35e-f90d-4ae8-9254-feacef0b9c3b                                                                     │
│  Agent: Financial Sentiment Reporter                                                                            │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 9cf35596-dcdb-4f2d-b439-9b7a329cb2b2                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯



*******************************************
   SENTIMENT ANALYSIS CREW FINISHED WORK!   
*******************************************


In [17]:
# @title 8. Display final sentiment brief

print("\n\n###############################################")
print(f"## AI-Assisted News Sentiment Brief for {stock_ticker}:")
print("## REMINDER: FOR INFORMATIONAL PURPOSES ONLY - NOT FINANCIAL ADVICE")
print("###############################################\n")

if 'result' in locals():
    # Print the final output from the 'sentiment_brief_writer' agent
    print(result)
else:
    print("!!! The 'result' variable is not defined. Ensure Cell 7 executed correctly.")

# Add final disclaimer again for safety
print("\n\n"+"*"*70)
print("!!! IMPORTANT DISCLAIMER !!!")
print("The above report is generated by an AI assistant for informational purposes ONLY.")
print("It is based on automated analysis of recent news and may contain errors or")
print("misinterpretations. Market conditions and sentiment can change rapidly.")
print("This output DOES NOT constitute financial or investment advice.")
print("Consult with a qualified financial advisor and conduct your own thorough")
print("research before making any investment decisions.")
print("Do not rely solely on this information.")
print("*"*70)



###############################################
## AI-Assisted News Sentiment Brief for NVDA:
## REMINDER: FOR INFORMATIONAL PURPOSES ONLY - NOT FINANCIAL ADVICE
###############################################

**NVDA Sentiment Analysis Report**

**Overall Sentiment Assessment**: Mixed/Neutral

**Article Sentiment Count**: 
- Positive: 1
- Neutral: 3
- Negative: 3

**Key Themes Driving Sentiment**:
1. **Market Performance Concerns**: Negative sentiment driven by NVDA's struggles in a challenging market environment, impacting stock performance due to broader market trends and potential internal company issues.
2. **Investor Caution**: Negative sentiment highlighted by cautious stances from investors and analysts, with recommendations to hold rather than buy NVDA stock, amid concerns over future product demand.
3. **Potential for Future Growth**: Mixed sentiment (Positive and Neutral) with some forecasts suggesting NVDA may outperform the market in the future, indicating confidence in 